In [51]:
# %pip freeze > requirements.txt 

# SparkContext and RDD basics



Spark revolves around the concept of a resilient distributed dataset (RDD), which is a fault-tolerant collection of elements that can be operated on in parallel. There are two ways to create RDDs: parallelizing an existing collection in your driver program, or referencing a dataset in an external storage system, such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat.

### Import libraries

In [52]:
# %pip install numpy

In [53]:
from pyspark import SparkContext
import numpy as np

## Initialize a `SparkContext` (the main abstraction to the cluster)
**Note the '4' in the argument. It denotes 4 cores to be used for this SparkContext object.**

In [54]:
# sc.stop() # stop the current SparkContext 

In [55]:
sc=SparkContext(master="local[4]")  

22/10/12 11:39:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [56]:
print(sc) #  print the SparkContext object

<SparkContext master=local[4] appName=pyspark-shell>


### Generate a list of random integeres

In [57]:
lst=np.random.randint(0,10,20) # starting 0 ,randomly select 20 elements up to 9

In [58]:
print(lst)

[8 2 0 3 0 2 6 2 0 1 6 8 1 1 6 6 0 7 0 3]


### Parallelize the list - this is the main operation toward distributed computing

In [59]:
A=sc.parallelize(lst) #parallelize the list to create an RDD object 

### What did we just do? We created a RDD? What is a RDD?
![](https://i.stack.imgur.com/cwrMN.png)

Spark revolves around the concept of a resilient distributed dataset (RDD), which is a **fault-tolerant collection of elements that can be operated on in parallel**. SparkContext manages the distributed data over the worker nodes through the cluster manager. 

There are two ways to create RDDs: 
* parallelizing an existing collection in your driver program, or 
* referencing a dataset in an external storage system, such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat.

We created a RDD using the former approach

### `A` is a pyspark RDD object, we cannot access the elements directly

In [60]:
type(A)

pyspark.rdd.RDD

In [61]:
A

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

### Opposite to parallelization - `collect` brings all the distributed elements and returns them to the head node. <br><br>Note - this is a slow process, do not use it often. 

In [62]:
A.collect() #collect the RDD object to a list   , collect is slow operation because it moves data from cluster to driver node (head node)  

[8, 2, 0, 3, 0, 2, 6, 2, 0, 1, 6, 8, 1, 1, 6, 6, 0, 7, 0, 3]

### How were the partitions created? Use `glom` method

In [63]:
A.glom().collect()  #glom() is used to collect the elements of each partition into a list

[[8, 2, 0, 3, 0], [2, 6, 2, 0, 1], [6, 8, 1, 1, 6], [6, 0, 7, 0, 3]]

### Now stop the SC and reinitialize it with 2 cores and see what happens when you repeat the process!

In [64]:
sc.stop()

In [65]:
sc=SparkContext(master="local[2]")

22/10/12 11:39:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [66]:
A = sc.parallelize(lst)

In [67]:
A.glom().collect()

#since only 2 cores are used, the elements are distributed in 2 partitions

[[8, 2, 0, 3, 0, 2, 6, 2, 0, 1], [6, 8, 1, 1, 6, 6, 0, 7, 0, 3]]

**The RDD is now distributed over two chunks, not four!** 

So, let's redo the process with 4 cores again.

In [68]:
sc.stop()

In [69]:
sc = SparkContext(master="local[4]")

22/10/12 11:39:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [70]:
A = sc.parallelize(lst)

## Basic operations
### `Count` the elements

In [71]:
A.count() #count the number of elements in the RDD object 

20

### The first element (`first`) and the first few elements (`take`)

In [72]:
A.first() #return the first element in the RDD object       

8

In [73]:
A.take(4) #return the first 4 elements in the RDD object , simialr to  head() in pandas

[8, 2, 0, 3]

### Removing duplicates: Get another RDD with only the `distinct` elements

The method `RDD.distinct()` Returns a new dataset that contains the distinct elements of the source dataset.

**NOTE**: This operation requires a **shuffle** in order to detect duplication across partitions. **So, it is a slow operation.**

In [74]:
A_distinct=A.distinct() # since RDD is immutable, we need to assign the distinct RDD object to a new variable

In [75]:
A_distinct.collect()

[8, 0, 1, 2, 6, 3, 7]

### To sum all the elements use `reduce` method

In [76]:
A.reduce(lambda x,y:x+y) #reduce the RDD object to a single value using a function  ((....((((((1+2)+3)+4)+5)+6)...))

62

### Or direct `sum` method

In [77]:
A.sum()

62

### Or using the `fold` method, which aggregates the elements of each partition, and then the results for all the partitions

In [78]:
A.fold(0,lambda x,y:x+y) #fold is similar to reduce, but it takes an initial value(0) as the first argument , 

62

### Finding maximum element by `reduce`

In [79]:
A.reduce(lambda x,y: x if x > y else y) #find the maximum element in the RDD object  .
# for [0, 4, 8, 1, 5, 6, 2, 7, 3 ]   steps is  first compare 0 and 4  as result is  4 , then compare 4 and 8 as result is 8 , then compare 8 and 1 as result is 8 , then compare 8 and 5 as result is 8 , then compare 8 and 6 as result is 8 , then compare 8 and 2 as result is 8 , then compare 8 and 7 as result is 8 , then compare 8 and 3 as result is 8 .. ( move  larger eelement to the x  and new element to y and again compare x and y until the last element)

8

### Finding longest word using `reduce`

In [80]:
words = 'These are some of the best Macintosh computers ever'.split(' ') #split the string into a list of words
wordRDD = sc.parallelize(words) #parallelize the list to create an RDD object
wordRDD.reduce(lambda w,v: w if len(w)>len(v) else v)  # as above lenght of first two words are compared and result is compared with the next word and so on

'computers'

## Functions/filtering over RDD
### Use `filter` to return a new RDD with elements satisfying a given predicate (lambda expression)

In [81]:
# Return RDD with elements divisible by 3
A.filter(lambda x:x%3==0).collect() # here we use collect() to form list for those element divisible by 3( after filtering)

[0, 3, 0, 6, 0, 6, 6, 6, 0, 0, 3]

### Lambda functions are short and sweet but we can write regular Python functions to use with `reduce`

In [82]:
def largerThan(x,y):
    """
    Returns the last word among the longest words in a list
    """
    if len(x)> len(y):
        return x
    elif len(y) > len(x):
        return y
    else: # if the lengths are equal, return the last word
        if x < y: return x 
        else: return y

    # len(x) > len(y) != x > y   because x > y  will return the first word if the length of the words are equal
    # x < y is used to return the last word if the length of the words are equal

In [83]:
largerThan('apple','banana')

'banana'

In [84]:
wordRDD.reduce(largerThan)

'Macintosh'

## Sampling an RDD
* RDDs are often very large.
* **Aggregates, such as averages, can be approximated efficiently by using a sample.** This comes handy often for operation with extremely large datasets where a sample can tell a lot about the pattern and descriptive statistics of the data.
* Sampling is done in parallel and requires limited computation.

The method `RDD.sample(withReplacement,p)` generates a sample of the elements of the RDD. where
- `withReplacement` is a boolean flag indicating whether or not a an element in the RDD can be sampled more than once.
- `p` is the probability of accepting each element into the sample. Note that as the sampling is performed independently in each partition, the number of elements in the sample changes from sample to sample.

In [85]:
# get a sample whose expected size is m
# Note that the size of the sample is different in different runs
m=5
n=20
print('sample1=',A.sample(False,m/n).collect()) 
print('sample2=',A.sample(False,m/n).collect())
print('sample3=',A.sample(False,m/n).collect())
print('sample4=',A.sample(False,m/n).collect())

sample1= [2, 0, 3, 2, 0, 6, 0]
sample2= [8, 6, 6, 7, 3]
sample3= [8, 2, 0, 6, 8, 1, 6, 0]
sample4= [0, 0, 0, 6, 1, 0]


### Things to note and think about
* Each time you run the previous cell, you get a different estimate
* The accuracy of the estimate is determined by the size of the sample $n*p$. Here, probability $p=\frac{m}{n}$
* See how the error changes as you vary $p$

## Basic statistics

In [86]:
print("Maximum: ",A.max())
print("Minimum: ",A.min())
print("Mean (average): ",A.mean())
print("Standard deviation: ",A.stdev())

Maximum:  8
Minimum:  0
Mean (average):  3.1
Standard deviation:  2.8442925306655784


In [87]:
A.stats()

(count: 20, mean: 3.1, stdev: 2.8442925306655784, max: 8.0, min: 0.0)

## Mapping
### `map` operation with _lambda_ function

In [88]:
B=A.map(lambda x:x*x)

In [89]:
B.collect()

[64, 4, 0, 9, 0, 4, 36, 4, 0, 1, 36, 64, 1, 1, 36, 36, 0, 49, 0, 9]

### `map` operation with regular Python function

In [90]:
def square_if_odd(x):
    if x%2==1:
        return x*x
    else:
        return x

In [91]:
A.map(square_if_odd).collect()

[8, 2, 0, 9, 0, 2, 6, 2, 0, 1, 6, 8, 1, 1, 6, 6, 0, 49, 0, 9]

### `flatmap` method returns a new RDD by first applying a function to all elements of this RDD, and then flattening the results

In [92]:
A.flatMap(lambda x:(x,x*x)).collect()  # here result of operation  is also flattened to a single list 

[8,
 64,
 2,
 4,
 0,
 0,
 3,
 9,
 0,
 0,
 2,
 4,
 6,
 36,
 2,
 4,
 0,
 0,
 1,
 1,
 6,
 36,
 8,
 64,
 1,
 1,
 1,
 1,
 6,
 36,
 6,
 36,
 0,
 0,
 7,
 49,
 0,
 0,
 3,
 9]

## Grouping and binning
### `groupby` returns a RDD of grouped elements (iterable) as per a given group operation (function)

In [93]:
result=A.groupBy(lambda x:x%2).collect()
print(A.collect())
print(sorted(result[0][1]))
sorted([(x, sorted(y)) for (x, y) in result])

[8, 2, 0, 3, 0, 2, 6, 2, 0, 1, 6, 8, 1, 1, 6, 6, 0, 7, 0, 3]
[0, 0, 0, 0, 0, 2, 2, 2, 6, 6, 6, 6, 8, 8]


[(0, [0, 0, 0, 0, 0, 2, 2, 2, 6, 6, 6, 6, 8, 8]), (1, [1, 1, 1, 3, 3, 7])]

### `histogram` method takes a list of bins/buckets and returns a tuple with result of the histogram (binning) 

In [94]:
A.histogram([x for x in range(0,100,10)])

([0, 10, 20, 30, 40, 50, 60, 70, 80, 90], [20, 0, 0, 0, 0, 0, 0, 0, 0])

## Set operations
### Create smaller RDDs to demonstrate joint operations

In [95]:
lst1=np.random.randint(0,10,3)
C=sc.parallelize(lst1)
lst2=np.random.randint(10,20,3)
D=sc.parallelize(lst2)
print("C:",C.collect())
print("D:",D.collect())

C: [3, 4, 2]
D: [10, 12, 18]


### `C+D` gives the union (like set union), not the element wise sum

In [96]:
(C+D).collect()

[3, 4, 2, 10, 12, 18]

### `cartesian` gives the pairwise product (as tuples) 

In [97]:
C.cartesian(D).collect()

[(3, 10),
 (3, 12),
 (3, 18),
 (4, 10),
 (4, 12),
 (4, 18),
 (2, 10),
 (2, 12),
 (2, 18)]

### `intersection` and `subtract `methods return a RDD of the set intersection and subtraction (difference)

In [98]:
rdd1 = sc.parallelize([1, 10, 2, 3, 4, 5])
rdd2 = sc.parallelize([1, 6, 2, 3, 7, 8])
rdd1.intersection(rdd2).collect()

[1, 2, 3]

In [99]:
rdd1.subtract(rdd2).collect()

[10, 4, 5]

### Stop the `SparkContext` at the end

In [100]:
sc.stop()